In [ ]:
from io import StringIO

import matplotlib.pyplot as plt
from scipy.io import arff
import seaborn as sns
from loguru import logger
import yaml

from datetime import datetime
import polars as pl
import pandas as pd
import numpy as np
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

# MODEL
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, LSTM, Dense, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import plot_model
from sklearn.preprocessing import (
    LabelEncoder, 
    StandardScaler,
    label_binarize
)
from sklearn.metrics import (
    classification_report, 
    confusion_matrix, 
    accuracy_score,
    precision_recall_fscore_support,
    balanced_accuracy_score,
    roc_auc_score,
    roc_curve
)

# Save variables for model
import joblib
import json

import tensorflow as tf
from sklearn.metrics import accuracy_score, precision_recall_fscore_support
from tensorflow.python.framework.convert_to_constants import convert_variables_to_constants_v2

# PERSONAL FUNCTIONS
from utils import *
from models.main import *
from models.optimizer import ViterbiLiteDecoder

from functions.windows import create_feature_windows # creación de ventanas e ingenieria de características
from functions.multimodal import create_multimodal_windows_robust

In [2]:
dict_preds = {
    'Eat':      {'y': [], 'y_pred': [], 'y_pred_probs': []},
    'Write':    {'y': [], 'y_pred': [], 'y_pred_probs': []},
    # 'Others':    {'y': [], 'y_pred': [], 'y_pred_probs': []} ,
    'Walk':     {'y': [], 'y_pred': [], 'y_pred_probs': []},
    'Sit':      {'y': [], 'y_pred': [], 'y_pred_probs': []},
    'Type':     {'y': [], 'y_pred': [], 'y_pred_probs': []},
}

In [3]:
dic_result = {
    'Eat':      {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}, 
    'Write':    {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
    'Walk':     {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
    'Sit':      {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
    # 'Others':    {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []},
    'Type':     {'Accuracy' : [], 'F1-score': [], 'Precision': [], 'Recall': []}
}

In [4]:
loaded = tf.saved_model.load(r"..\cnn_tcn\model_with_stand_in_others\saved_model")
infer = loaded.signatures["serving_default"]
label_encoder = joblib.load(r'..\cnn_tcn\model_with_stand_in_others\label_encoder.joblib')

In [5]:
path_base = r"..\data\real-data"
archive_date = [archive for archive in os.listdir(path_base) if '.json' not in archive]


for data in archive_date:
    path_data = os.path.join(path_base, data)
    file_data = os.listdir(path_data)

    target = data.split('-')[0].title()
        
    if target == 'Stand' or target == 'Sensor' or target == 'Day': #or target == "Write":
        continue
    
    for file in file_data:

        file_path = os.path.join(path_data, file)
        
        with open(file_path, 'rb') as file:
            data = json.load(file)

        gyro_df = data['gyro']
        accel_df = data['accel']

        accel_temp = pl.DataFrame(accel_df)
        gyro_temp = pl.DataFrame(gyro_df)

        
        accel_temp = accel_temp.with_columns(pl.lit('A').alias('Usuario'))
        gyro_temp  = gyro_temp.with_columns(pl.lit('A').alias('Usuario'))

        accel_temp = accel_temp.with_columns(pl.lit(target).alias('gt'))
        gyro_temp = gyro_temp.with_columns(pl.lit(target).alias('gt'))

        df_accel = normalize_columns(accel_temp,
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )

        df_gyro = normalize_columns(gyro_temp, 
                            user_col_name  = "Usuario", 
                            timestamp_col_name = "timestamp", 
                            label_col_name = "gt", 
                            x_col_name = "x", 
                            y_col_name = "y", 
                            z_col_name = "z"
                        )
        
        df_accel = convert_timestamp(df_accel)
        df_gyro = convert_timestamp(df_gyro)

        X_all, y_all, subjects_all, metadata_all = create_multimodal_windows_robust(
            df_accel = df_accel, 
            # df_gyro = df_gyro,
            window_seconds=5,
            overlap_percent=50,
            sampling_rate=20,
            target_timesteps=100,
            min_data_threshold=0.8,  # 80% mínimo de datos
            max_gap_seconds=1.0      # Máximo 1 segundo de gap
        )

        X_tensor = tf.constant(X_all, dtype=tf.float32)
        y_pred = infer(X_tensor)
        y_all = label_encoder.transform(y_all)

        # Ejemplo: y_pred con probabilidades
        y_pred_probs = list(y_pred.values())[0].numpy()

        # Convertir a clases predichas (índice del máximo)
        y_pred_classes = np.argmax(y_pred_probs, axis=1)


        # Comparar con y_real (asegúrate de que y_real tenga etiquetas 0..N-1)
        acc = accuracy_score(y_all, y_pred_classes)
        
        precision, recall, f1, _ = precision_recall_fscore_support(
                y_all, y_pred_classes, average='macro', zero_division=0
            )
        
        dic_result[target]['Accuracy'].append(acc*100)
        dic_result[target]['F1-score'].append(f1*100)
        dic_result[target]['Precision'].append(precision*100)
        dic_result[target]['Recall'].append(recall*100)

        dict_preds[target]['y'].extend(y_all)
        dict_preds[target]['y_pred'].extend(y_pred_classes) 

🔧 CONFIGURACIÓN MONOMODAL (Solo Accel):
  Duración: 5s
  Timesteps objetivo: 100
  Canales objetivo: 3
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Tolerancia sincronización: 50ms
  ACCEL preparado: 6531 muestras

📏 PARÁMETROS TEMPORALES:
  Duración de ventana: 5s
  Paso entre ventanas: 2.50s

👤 Usuario A, Eat:
   Accel: 6531 muestras
   ✅ Creadas 59 ventanas válidas

📊 RESUMEN MULTIMODAL:
  Ventanas intentadas: 59
  Ventanas creadas: 59
  Ventanas con gyro: 0
  Ventanas solo accel: 59
  Tasa de éxito: 100.0%

✅ RESULTADO FINAL MULTIMODAL:
  Forma de X: (59, 100, 3) (samples, timesteps, channels)
  Canales: 3 (solo accel_xyz)
  Total ventanas: 59
  Usuarios únicos: 1
  Actividades: ['Eat']
🔧 CONFIGURACIÓN MONOMODAL (Solo Accel):
  Duración: 5s
  Timesteps objetivo: 100
  Canales objetivo: 3
  Frecuencia de muestreo: 20Hz
  Solapamiento: 50%
  Umbral mínimo de datos: 80.0%
  Máximo gap permitido: 1.0s
  Tolerancia sinc

In [6]:
from sklearn.metrics import classification_report

y_true_global = []
y_pred_global = []
y_pred_probs_global = []

for activity in dict_preds.keys():
    y_true_global.extend(dict_preds[activity]['y'])
    y_pred_global.extend(dict_preds[activity]['y_pred'])
    y_pred_probs_global.extend(dict_preds[activity]['y_pred_probs'])

labels = np.unique(y_true_global)  # clases presentes en los datos
target_names = label_encoder.inverse_transform(labels)

print(classification_report(
    y_true_global, y_pred_global,
    labels=labels,
    target_names=target_names
))

              precision    recall  f1-score   support

         Eat       0.70      0.64      0.67       472
         Sit       0.75      0.82      0.79       485
        Type       0.95      0.88      0.92       537
        Walk       0.98      0.94      0.96       619
       Write       0.99      0.98      0.98       275

   micro avg       0.87      0.85      0.86      2388
   macro avg       0.87      0.85      0.86      2388
weighted avg       0.87      0.85      0.86      2388



In [7]:
acc_global = accuracy_score(y_true_global, y_pred_global)
print(f'Accuracy global: {acc_global*100:.2f}%')

Accuracy global: 84.63%
